In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

writer=csv.writer(open('test.csv','w',newline=''))

#URL raíz desde donde se obtendran todos los datos
URL = 'https://seer.cancer.gov/statfacts/more.html'

#Obtener código fuente página web
page = requests.get(URL)

#Interpretar oagina web para obetneer sus elementos
soup = BeautifulSoup(page.content, 'html.parser')

#Encontrar el elemento donde se listan los tipos de cancer.
results = soup.find('div', class_='alphaList rpw')

#Obtener los link a las páginas de los diferentes tipos de cancer
link  = {}
for job_elem in results.find_all('ul'):
    for job_elem2 in job_elem.find_all('a'):
        link[job_elem2.get_text()] = job_elem2.get('href')    
 
#declaración de diccionario para almacenar datos
datos_cancer = {}

#cabecera del archivo csv
cabecera = ['tipo_cancer',
            'anio',
            'nuevos_casos_SEER9_obs',
            'nuevos_casos_SEER9_ten',
            'nuevos_casos_SEER13_obs',
            'nuevos_casos_SEER13_ten',
            'muertes_obs',
            'muertes_ten',
            'por_sup_5_anios_obs',
            'por_sup_5_anios_ten']
writer.writerow(cabecera)

#recorrer los tipos de cancer
for tipo_cancer in link.keys():
    
    #ignorar tipos que no contienen datos
    if tipo_cancer == "All Cancers" or \
               tipo_cancer == "Female Breast Subtypes" or \
               tipo_cancer == "Cancer Disparities" or \
               tipo_cancer == "Common Cancer Sites":
               continue

    #Obtener las paginas de los tipos de datos
    data = link.get(tipo_cancer, "")
    url2 = 'https://seer.cancer.gov'+data
    page1 = requests.get(url2)
    soup1 = BeautifulSoup(page1.content, 'html.parser')

    #Obtener filas de las tablas de datos   
    results3 = soup1.find(id = 'scrapeTable_01')
    filas = results3.find_all('tr')

    #Recorrer columnas de filas de las tablas de datos
    tabla = []
    for fila in filas:
        fila_temp = []
        fila_temp.append(tipo_cancer)
        columnas = fila.find_all('td')

        if len(columnas) != 0:
            for tds in fila.find_all('td'):
                
                #Obtener el texto de cada campo
                valor = tds.get_text()
                
                #Remover caracteres extraños
                if "%" in valor:
                    valor = valor[0:-1] if valor[-1] == '%' else valor
                elif "-" in valor:
                    valor = ""
                    
                #Crear lista con valores de fila    
                fila_temp.append(valor)
            
            #Escribir a csv
            writer.writerow(fila_temp)

        tabla.append(fila_temp)
    datos_cancer[tipo_cancer] = tabla


